In [1]:
import datetime as dt
import time
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

#import pyxu.abc as pxa
#import pyxu.operator as pxop
import pyxu.opt.stop as pxos
#from pyxu.opt.solver.pgd import PGD

import pfw_hawkes as pyfwl
from hawkes_likelihood import HawkesLikelihood

seed = 1  # None  # for reproducibility
if seed is None:
    seed = np.random.randint(0, 1000)
print(f"Seed: {seed}")

Seed: 1


In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib qt

In [3]:
# Parameter for reconstruction
#lambda_factor = 0.1

## Parameters of the solvers
# Base
remove = True
min_iterations = 1
# Stop
eps = 1e-4
tmax = 15.0
eps_dcv = 1e-2
# PFW
ms_threshold = 0.8
init_correction = 1e-1
final_correction = 1e-6
correction_steps = 5

### Generation of the source

In [4]:
path = "simulated_data/events0.csv"
hpL = HawkesLikelihood(path=path)

In [42]:
start = time.time()
lip = hpL.opA.estimate_lipschitz(method='svd')
print("Computation of the Lipschitz constant: {:.4f}".format(time.time()-start))
#np.linalg.svd to compute the largest singular value which is = to the lipschitz constant for linear operators

Computation of the Lipschitz constant: 0.0413


In [44]:
import pyxu

In [46]:
print(hpL.opA.shape)

(1809, 72)


In [45]:
pyxu.__version__

'1.1.0'

In [6]:
#lambda_ = lambda_factor * np.linalg.norm(op.T(measurements), np.infty)  # rule of thumb to define lambda
lambda_ = 10e-5

### Defining solvers

In [7]:
pfw = pyfwl.PFWLasso(
    hpL,  # replace by likelihood operator hpL
    hpL.opA,  # replace by hpL.opA
    lambda_,
    ms_threshold=ms_threshold,
    init_correction_prec=init_correction,
    final_correction_prec=final_correction,
    remove_positions=remove,
    min_correction_steps=correction_steps,
    show_progress=False,
)

In [8]:
stop_crit = pxos.RelError(
    eps=eps,
    var="objective_func",
    f=None,
    norm=2,
    satisfy_all=True,
)
# alternative stopping criteria
dcv = pyfwl.dcvStoppingCrit(eps_dcv)

# Minimum number of iterations
min_iter = pxos.MaxIter(n=min_iterations)

# Maximum duration
max_duration = pxos.MaxDuration(t=dt.timedelta(seconds=tmax))

stop = (min_iter & stop_crit) | max_duration

# track DCV
track_dcv = pxos.AbsError(eps=1e-10, var="dcv", f=None, norm=2, satisfy_all=True)

### Solving

In [9]:
print("Polyatomic FW: Solving ...")
start = time.time()
pfw.fit(stop_crit= stop | track_dcv, diff_lipschitz=lip**2)
data_p, hist_p = pfw.stats()
time_p = time.time() - start
print("\tSolved in {:.3f} seconds".format(time_p))


Polyatomic FW: Solving ...
diff_lipschitz constant provided.
	Solved in 0.012 seconds


### Evaluation of the solve

In [10]:
print("Final value of dual certificate:\n\tPFW: {:.4f}".format(data_p["dcv"]))
print(
    "Final value of objective function:\n\tPFW : {:.2f}".format(hist_p[-1][-1]))

Final value of dual certificate:
	PFW: -316752137604238.8750
Final value of objective function:
	PFW : 37322.60


In [11]:
print(data_p["x"])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
